In [416]:
##1. Imports
##Placing all the imports at the beginning of the notebook
import pandas as pd


1.2 Objectives 
a. Goal is to find out how many total regions are there?
b. How many sites are there ?
c. How many sites per Region are there?
d. We are going to build one timeforecasting model per site?
e. Find Null values per Region per site.


In [417]:
## 2. Data Collection

# Load the data from the saved local directory in csv format

usa_camping_data_23 = pd.read_csv('/Users/priyankapanhalkar/Documents/camping/reservation_2023.csv')


/var/folders/ls/mlx1cxs96mx4gpcd6hv_g4f00000gn/T/ipykernel_35812/3725128745.py:5: DtypeWarning: Columns (7,11,12,13,15,17,27,29,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  usa_camping_data_23 = pd.read_csv('/Users/priyankapanhalkar/Documents/camping/reservation_2023.csv')


In [418]:
## 3. Data Definition

#Call the info method on camping_data_23 to see a summary of the data


usa_camping_data_23.info()
usa_camping_data_23.describe()
usa_camping_data_23.head(5)

#To find the names of all columns due to truncated results
selected_columns = usa_camping_data_23.iloc[:, [14,15,16,17,18,19, 20,21,22,23,24,25,26,27,28,29,30,31,32,33,34]]
selected_columns.head(2)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8984097 entries, 0 to 8984096
Data columns (total 35 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   historicalreservationid  object 
 1   ordernumber              object 
 2   agency                   object 
 3   orgid                    int64  
 4   codehierarchy            object 
 5   regioncode               object 
 6   regiondescription        object 
 7   parentlocationid         object 
 8   parentlocation           object 
 9   legacyfacilityid         float64
 10  park                     object 
 11  sitetype                 object 
 12  usetype                  object 
 13  productid                object 
 14  inventorytype            object 
 15  facilityid               object 
 16  facilityzip              float64
 17  facilitystate            object 
 18  facilitylongitude        float64
 19  facilitylatitude         float64
 20  customerzip              object 
 21  tax     

,inventorytype,facilityid,facilityzip,facilitystate,facilitylongitude,facilitylatitude,customerzip,tax,usefee,tranfee,...,totalbeforetax,discount,totalpaid,startdate,enddate,orderdate,nights,numberofpeople,equipmentdescription,equipmentlength
0,CAMPING,10049834,NaN,NaN,NaN,NaN,20001,0.0,NaN,NaN,...,0.0,0.0,0.0,2023-01-08T00:00:00Z,2023-01-09T00:00:00Z,2023-01-04T14:02:43.189398Z,1 day,1.0,Tent,0.0
1,CAMPING,10049834,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,2023-05-01T00:00:00Z,2023-05-08T00:00:00Z,2023-04-28T16:13:47.462979Z,7 days,1.0,Tent,0.0


In [419]:
#What columns are categorical and what are measureable
categorical_columns = usa_camping_data_23.select_dtypes(include=['object', 'category']).columns
print("Categorical columns:", categorical_columns)

Categorical columns: Index(['historicalreservationid', 'ordernumber', 'agency', 'codehierarchy',
       'regioncode', 'regiondescription', 'parentlocationid', 'parentlocation',
       'park', 'sitetype', 'usetype', 'productid', 'inventorytype',
       'facilityid', 'facilitystate', 'customerzip', 'totalpaid', 'startdate',
       'enddate', 'orderdate', 'nights', 'equipmentdescription'],
      dtype='object')


In [420]:
non_categorical_columns = usa_camping_data_23.select_dtypes(exclude=['object', 'category']).columns
print(non_categorical_columns)  

Index(['orgid', 'legacyfacilityid', 'facilityzip', 'facilitylongitude',
       'facilitylatitude', 'tax', 'usefee', 'tranfee', 'attrfee',
       'totalbeforetax', 'discount', 'numberofpeople', 'equipmentlength'],
      dtype='object')


3.1 Find how many unique regioncodes, region and sites etc are there in the dataset


In [421]:
usa_camping_data_23.regioncode.unique()
unique_regioncode = len(usa_camping_data_23['regioncode'].unique())
print(unique_regioncode)

47


In [422]:
usa_camping_data_23.regiondescription.unique()
unique_regiondescription = len(usa_camping_data_23['regiondescription'].unique())
print(unique_regiondescription)

46


In [423]:
usa_camping_data_23.parentlocationid.unique()
unique_parentlocationid = len(usa_camping_data_23['parentlocationid'].unique())
print(unique_parentlocationid)

1077


In [424]:
usa_camping_data_23.parentlocation.unique()
unique_parentlocation = len(usa_camping_data_23['parentlocation'].unique())
print(unique_parentlocation)

575


In [425]:
usa_camping_data_23.facilityid.unique()
unique_facilityid = len(usa_camping_data_23['facilityid'].unique())
print(unique_facilityid)

4744


a. There are 47 unique regioncodes
b. There are 46 unique regiondescriptions
c. There are 1077 unique parentlocationids
d. There are 575 unique parentlocations
e. there are 4744 unique facility Ids
f. There are 22 categorical 13 non-categorical columns


Find the null counts for each column

In [426]:
#Since Parentlocationid and parentlocation are not in consensus lets Find the null values in both to understand better whats is happening.
null_count_parentlocation = usa_camping_data_23['parentlocation'].isnull().sum()
print(null_count_parentlocation)

26


In [427]:
null_count_parentlocationid = usa_camping_data_23['parentlocationid'].isnull().sum()
print(null_count_parentlocationid)

16


Conclusion- Each parent location has multiple parent locations IDs. Also not all Parent location IDs are integers.

In [428]:
#Print rows where parentlocationID is missing
missing_parentlocationid_df=usa_camping_data_23[usa_camping_data_23['parentlocationid'].isnull()]
print(missing_parentlocationid_df)


                     historicalreservationid   ordernumber agency  orgid  \
610236  38727093-b2d6-473f-972a-67db3132f73f  0501282199-1    BLM    126   
610237  428ca20a-c350-4657-9c4c-5658a785ad1e  0506344544-1    BLM    126   
610238  5728a53f-469c-49e5-aa25-649da64656f8  0509217089-2    BLM    126   
610239  c893a876-c164-40ce-bef0-fb14152b290f  0511871934-1    BLM    126   
610240  02327aa0-c883-4ea1-8374-c7c5453b55d1  0517157469-1    BLM    126   
610241  303b347f-17cc-44c0-bdc6-5c2aff0affa1  0521239847-1    BLM    126   
610242  ed3207cf-e0a5-4f98-a94f-5be44d33a731  0529915109-1    BLM    126   
610243  b19161f4-58d7-4bb6-998c-0f267b06f382  0537683580-1    BLM    126   
610244  89e673b3-9256-4800-924f-3567a4ba08e9  0539578369-1    BLM    126   
610245  68807b36-9f70-4c22-af77-3a7c12df7edd  0551871618-1    BLM    126   
610246  210ba525-e581-4bea-a42b-d8bfbb58277f  0553484798-1    BLM    126   
610247  783e2190-a58f-4a73-abf5-9bb421d2a272  0554220472-1    BLM    126   
610248  13c7

In [429]:
grouped_counts = usa_camping_data_23.groupby(['parentlocationid','parentlocation']).size()
print(grouped_counts)

parentlocationid                      parentlocation                           
141.0                                 Dalton Highway                                 29
202                                   Barre Falls Dam                                24
219                                   North Springfield Lake                         14
221                                   Otter Brook Lake                               63
222                                   Surry Mountain Lake                            57
                                                                                   ... 
b142b0c4-8d63-44de-9076-d55c419c2d7f  Mississippi River Headwaters - Gull Lake     2330
b39eaf81-4006-4265-9208-3cdef0708615  Arkansas River - Wilbur D. Mills Pool        2734
b6279382-134d-4ab9-9571-22114763d09a  Arkansas River - Pool 3                       706
e5052851-190b-470a-a9f2-80a0f18c08eb  Mississippi River Headwaters - Sandy Lake    1993
e61c99cf-fc25-4ae3-b7d8-9707b2198f9f  Ar

In [430]:
sorted_grouped_counts = pd.DataFrame(grouped_counts).sort_values(by=['parentlocationid','parentlocation'], ascending=[False,True])
print(sorted_grouped_counts)

                                                                                   0
parentlocationid                     parentlocation                                 
e61c99cf-fc25-4ae3-b7d8-9707b2198f9f Arkansas River - Terry Lock and Dam        3105
e5052851-190b-470a-a9f2-80a0f18c08eb Mississippi River Headwaters - Sandy Lake  1993
b6279382-134d-4ab9-9571-22114763d09a Arkansas River - Pool 3                     706
b39eaf81-4006-4265-9208-3cdef0708615 Arkansas River - Wilbur D. Mills Pool      2734
b142b0c4-8d63-44de-9076-d55c419c2d7f Mississippi River Headwaters - Gull Lake   2330
...                                                                              ...
222                                  Surry Mountain Lake                          57
221                                  Otter Brook Lake                             63
219                                  North Springfield Lake                       14
202                                  Barre Falls Dam             

In [431]:
grouped_counts_parentlocation_by_parentlocationid = usa_camping_data_23.groupby(['parentlocation','parentlocationid']).size()
print(grouped_counts_parentlocation_by_parentlocationid)

parentlocation                           parentlocationid
18 Road - North Fruita Desert            16582                 5016
Abiquiu Lake                             74592                 4411
Acadia National Park                     74271               230746
Adams National Historical Park           2555                  2070
African Burial Ground National Monument  13906                  521
                                                              ...  
Yellowstone National Park                72451                43883
Yosemite National Park                   74296               271528
Youghiogheny River Lake                  74564                 3229
Zapata Falls SRMA                        16587                 4232
Zion National Park                       74297               315116
Length: 1086, dtype: int64


In [432]:
#Find relation between parentlocationIds and Region
grouped_counts_parentlocationID_by_region = usa_camping_data_23.groupby(['regiondescription','parentlocationid']).size()
print(grouped_counts_parentlocationID_by_region)

regiondescription  parentlocationid
Alaska             10142201             258
                   72195               1415
Alaska Region      2986                  13
                   70712                 77
                   72496                151
                                       ... 
Utah               10008689             451
                   10008692              39
                   10041073             394
Washington         72483                  1
                   72484               2050
Length: 1055, dtype: int64


In [433]:
#Find relation between parentlocations and Region
grouped_counts_parentlocation_by_region = usa_camping_data_23.groupby(['regiondescription','parentlocation']).size()
print(grouped_counts_parentlocation_by_region)

regiondescription  parentlocation                          
Alaska             BLM Campbell Creek Science Center             258
                   White Mountains National Recreation Area     1415
Alaska Region      Chugach National Forest                     14407
                   Katmai National Park & Preserve              2595
                   Kenai Fjords National Park                    151
                                                               ...  
Utah               Sand Flats Recreation Area                    680
                   Virgin River Canyon Recreation Area (P)      2089
                   Westwater Canyon River                       2049
Washington         Spokane District Office                         1
                   Yakima River Canyon                          2050
Length: 565, dtype: int64


In [434]:
#Find relation between parentlocationIds and facilityid
grouped_counts_parentlocationID_by_facilityid = usa_camping_data_23.groupby(['facilityid','parentlocationid']).size()
print(grouped_counts_parentlocationID_by_facilityid)

facilityid                            parentlocationid
1102                                  74313                 13
1826                                  74976                442
2264                                  74361                 62
16221                                 74980                976
16851                                 16221               1415
                                                          ... 
fd3951a5-a4cb-11ec-a5cd-b22fe006a94c  74646                311
fdc42ec5-213c-11ec-8d40-1aa8b530b339  74828                 50
fdf08bf1-ec5a-11ea-b83f-6e0e43d29f74  74447                434
fea6aa00-f6a7-11ea-ab86-ea94e31cc891  74577                114
ff9de9f8-ea5d-11eb-a9f1-725a14274964  16511                  9
Length: 4759, dtype: int64


In [435]:
#Find relation between   facilityid  and parentlocationIds
grouped_counts_facilityid_by_parentlocationid = usa_camping_data_23.groupby(['parentlocationid','facilityid']).size()
print(grouped_counts_facilityid_by_parentlocationid)

parentlocationid                      facilityid
141.0                                 10191011        26
                                      10276314         3
202                                   10048970        24
219                                   249983          14
221                                   249980          63
                                                    ... 
b39eaf81-4006-4265-9208-3cdef0708615  232744         374
b6279382-134d-4ab9-9571-22114763d09a  232688         695
                                      234242          11
e5052851-190b-470a-a9f2-80a0f18c08eb  233620        1993
e61c99cf-fc25-4ae3-b7d8-9707b2198f9f  232746        3105
Length: 4759, dtype: int64


3.2 Find how many  unique sitetype, usetype, productid, inventorytype, facilityid, facilitystate, parentlocations,parentlocationIds  are there in the dataset
a. There are 439 sitetypes
b. There are 5 usetypes
c. There are 94584 productids
d. There are 14 inventorytypes
e.There are 4744 facilityids
f.There are 50 facilitystates
g. There are 16 equipmentdescriptions
h.There are 575  parentlocations
i.There are 1077 parentlocationsIDs

In [436]:
usa_camping_data_23.legacyfacilityid.unique()
usa_camping_data_23.legacyfacilityid.unique()
unique_legacyfacilityid = len(usa_camping_data_23['legacyfacilityid'].unique())
print(unique_legacyfacilityid)

3726


In [437]:
usa_camping_data_23.parentlocation.unique()
usa_camping_data_23.parentlocation.unique()
unique_parentlocation = len(usa_camping_data_23['parentlocation'].unique())
print(unique_parentlocation)

575


In [438]:
usa_camping_data_23.parentlocationid.unique()
unique_parentlocationid = len(usa_camping_data_23['parentlocationid'].unique())
print(unique_parentlocationid)

1077


In [439]:
usa_camping_data_23.sitetype.unique()
usa_camping_data_23.sitetype.unique()
unique_sitetype = len(usa_camping_data_23['sitetype'].unique())
print(unique_sitetype)

439


In [440]:
usa_camping_data_23.usetype.unique()
unique_usetype = len(usa_camping_data_23['usetype'].unique())
print(unique_usetype)

5


In [441]:
usa_camping_data_23.productid.unique()
unique_productid = len(usa_camping_data_23['productid'].unique())
print(unique_productid)

94584


In [442]:
usa_camping_data_23.inventorytype.unique()
unique_inventorytype = len(usa_camping_data_23['inventorytype'].unique())
print(unique_inventorytype)

14


In [443]:
usa_camping_data_23.facilityid.unique()
unique_facilityid = len(usa_camping_data_23['facilityid'].unique())
print(unique_facilityid)

4744


In [444]:
usa_camping_data_23.facilitystate.unique()
unique_facilitystate = len(usa_camping_data_23['facilitystate'].unique())
print(unique_facilitystate)

50


In [445]:
usa_camping_data_23.equipmentdescription.unique()
unique_equipmentdescription = len(usa_camping_data_23['equipmentdescription'].unique())
print(unique_equipmentdescription)

16


#### Number of Missing Values in the entire dataset


1. Find the percentage of Null Values in Regioncode, Regiondescription and SiteType  , Inventory type, equipment description


In [446]:
# Calculate the Percentage of null values in dataset
null_percentage = usa_camping_data_23.isnull().mean() * 100
print (null_percentage)

historicalreservationid      0.000000
ordernumber                  0.000000
agency                       0.000000
orgid                        0.000000
codehierarchy                0.000000
regioncode                   0.168197
regiondescription            0.168197
parentlocationid             0.000178
parentlocation               0.000289
legacyfacilityid            39.793159
park                         0.000000
sitetype                    36.533455
usetype                     51.327807
productid                    0.000000
inventorytype                0.000000
facilityid                   0.000000
facilityzip                 22.977167
facilitystate               22.822372
facilitylongitude           18.694901
facilitylatitude            18.694901
customerzip                 19.655876
tax                          0.000045
usefee                     100.000000
tranfee                    100.000000
attrfee                     99.999955
totalbeforetax               0.000000
discount    

In [447]:
sorted_data = null_percentage.sort_values(ascending=False)
print(sorted_data)

tranfee                    100.000000
usefee                     100.000000
attrfee                     99.999955
equipmentdescription        57.044275
equipmentlength             54.079681
usetype                     51.327807
legacyfacilityid            39.793159
sitetype                    36.533455
numberofpeople              35.360749
enddate                     31.648445
nights                      31.628766
facilityzip                 22.977167
facilitystate               22.822372
customerzip                 19.655876
facilitylongitude           18.694901
facilitylatitude            18.694901
regiondescription            0.168197
regioncode                   0.168197
parentlocation               0.000289
parentlocationid             0.000178
tax                          0.000045
orderdate                    0.000033
startdate                    0.000033
totalpaid                    0.000000
agency                       0.000000
orgid                        0.000000
codehierarch

Among the most required columns to answer our questions below are the columns which are missing most data
1. Sitetype - 36.5%
2. Usetype - 51.3%
3. Number of people - 35.3%
4. Equipment description - 57%

Among the most required columns to answer our questions below are the columns which have most data
1. Region code
2. Region Description
3. Parent location
4. Inventorytype

In [448]:
# Calculate sum of null values in 'regiondescription' column

null_count_regiondescription = usa_camping_data_23['regiondescription'].isnull().sum()
print (null_count_regiondescription)


# Calculate the Percentage of null values in regiondescription
null_percentage_regiondescription = (usa_camping_data_23['regiondescription'].isnull().mean()) * 100
print (null_percentage_regiondescription)


15111
0.1681972044602813


In [449]:
#Calculate sum of null values in 'regioncode' column

null_count_regioncode = usa_camping_data_23['regioncode'].isnull().sum()
print (null_count_regioncode)


# Calculate the Percentage of null values in regioncode
null_percentage_regioncode = (usa_camping_data_23['regioncode'].isnull().mean()) * 100
print (null_percentage_regioncode)

15111
0.1681972044602813


In [450]:
#Calculate sum of null values in 'sitetype' column

null_count_sitetype = usa_camping_data_23['sitetype'].isnull().sum()
print (null_count_sitetype)


# Calculate the Percentage of null values in sitetype
null_percentage_sitetype = (usa_camping_data_23['sitetype'].isnull().mean()) * 100
print (null_percentage_sitetype)

3282201
36.53345461430347


In [451]:
#null_percentage_by_region_inventorytype = usa_camping_data_23.groupby('regiondescription').apply(lambda x: x.apply(null_percentage))
#print(null_percentage_by_group)

def null_percentage(column):
    return column.isnull().mean() * 100

# Group by 'regiondescription' and calculate NULL percentage for 'inventorytype' column
null_percentage_by_region_inventorytype = usa_camping_data_23.groupby('regiondescription')['inventorytype'].apply(null_percentage)
print(null_percentage_by_region_inventorytype)

regiondescription
Alaska                                  0.0
Alaska Region                           0.0
Arizona                                 0.0
California                              0.0
Colorado                                0.0
Colorado (BLM)                          0.0
Eastern Region                          0.0
Great Lakes / Ohio River                0.0
Great Plains                            0.0
Idaho (BLM)                             0.0
Intermountain Region                    0.0
Jonestopia                              0.0
Mid Pacific                             0.0
Midwest Region                          0.0
Mississippi Valley                      0.0
Montana                                 0.0
Morale, Welfare & Recreation Lodging    0.0
Nara Region                             0.0
National Capitol Region                 0.0
Nevada                                  0.0
New Mexico (BLM)                        0.0
North Atlantic Division                 0.0
Northeast Regi

In [452]:
# Group by 'regiondescription' and calculate NULL percentage for 'sitetype' column
null_percentage_by_region_sitetype = usa_camping_data_23.groupby('regiondescription')['sitetype'].apply(null_percentage)
print(null_percentage_by_region_sitetype)

regiondescription
Alaska                                    2.809325
Alaska Region                             2.850597
Arizona                                  89.928807
California                               10.110926
Colorado                                  0.056968
Colorado (BLM)                            0.000000
Eastern Region                           30.123554
Great Lakes / Ohio River                  0.291783
Great Plains                              0.000000
Idaho (BLM)                               0.259965
Intermountain Region                     64.528080
Jonestopia                                6.818182
Mid Pacific                               0.052319
Midwest Region                            3.067787
Mississippi Valley                        0.330314
Montana                                   0.013046
Morale, Welfare & Recreation Lodging      0.037679
Nara Region                               0.000000
National Capitol Region                  74.769710
Nevada       

Above results are aligned with previous findings of mean missing values

In [453]:
#calculate  how many sites are associated with each region

grouped_sites_per_region = usa_camping_data_23.groupby(['regiondescription','sitetype']).size()
print(grouped_sites_per_region)

regiondescription  sitetype             
Alaska             CABIN NONELECTRIC        1415
                   Hike                       11
                   Historic Tour              67
                   Nature Tour               133
Alaska Region      CABIN NONELECTRIC        7965
                                            ... 
Utah               TENT ONLY NONELECTRIC     235
                   Trailhead                1940
Washington         STANDARD NONELECTRIC     1883
                   TENT ONLY NONELECTRIC      88
                   WALK TO                    79
Length: 964, dtype: int64


In [454]:
#calculate  how many equipmentdescription are associated with each region

grouped_equipmentdescription_per_region = usa_camping_data_23.groupby(['regiondescription','equipmentdescription']).size()
print(grouped_equipmentdescription_per_region)

regiondescription  equipmentdescription
Alaska Region      Boat                      70
                   Car                       10
                   Caravan/Camper Van        76
                   Fifth Wheel               15
                   Pickup Camper             62
                                           ... 
Utah               Trailer                 1821
                   Vehicle                 1449
Washington         RV                       395
                   Tent                    1019
                   Trailer                  636
Length: 413, dtype: int64


In [455]:
#calculate  how many regions are associated with each State

grouped_regions_per_state = usa_camping_data_23.groupby(['facilitystate','regiondescription']).size()
print(grouped_regions_per_state)

facilitystate  regiondescription      
Alabama        South Atlantic Division     47519
               Southern Region              6032
Alaska         Alaska                       1415
               Alaska Region               27348
               Region 7 Alaska              1320
                                           ...  
Wisconsin      Eastern Region               8940
               Midwest Region               2884
               Mississippi Valley           8498
Wyoming        Intermountain Region       149430
               Rocky Mountain Region       21491
Length: 160, dtype: int64


In [456]:
#calculate  how many facilityids are associated with each facilityState

grouped_facilityid_per_state = usa_camping_data_23.groupby(['facilitystate','facilityid']).size()
print(grouped_facilityid_per_state)

facilitystate  facilityid                          
Alabama        232423                                  1918
               232529                                  2142
               232548                                   659
               232568                                  1910
               232569                                  3912
                                                       ... 
Wyoming        264411                                    78
               84424ac2-c1fa-11ea-8524-2a3500cf4453    5195
               9b777f86-e645-11ea-8a7e-f638cd389a69    2494
               9ce25109-e642-11ea-bfd0-660a8c194aab     865
               a844e204-ebcb-11ea-8045-369699f3be2d    1816
Length: 4384, dtype: int64


In [457]:
#calculate  how many facilityids are associated with each Regiondescription
grouped_facilityid_per_region = usa_camping_data_23.groupby(['regiondescription','facilityid']).size()
print(grouped_facilityid_per_region)

regiondescription  facilityid                          
Alaska             252494                                  1415
                   10089159                                 198
                   10089669                                  60
Alaska Region      232337                                   645
                   232352                                   374
                                                           ... 
Utah               AP2476                                  2872
                   VR1400193                                 16
                   f3cc9e0c-c5e8-11ea-b6a5-2628f1866643    5119
Washington         15160                                      1
                   250985                                  2050
Length: 4698, dtype: int64


In [458]:
#calculate  how many regions are associated with each facilityID
grouped_facilityid_per_regiondescription= usa_camping_data_23.groupby(['facilityid','regiondescription']).size()
print(grouped_facilityid_per_regiondescription)

facilityid                            regiondescription       
1102                                  Pacific Northwest Region      13
1826                                  Nevada                       442
2264                                  Southwestern Division         62
16221                                 Arizona                      976
16851                                 Arizona                     1415
                                                                  ... 
fd3951a5-a4cb-11ec-a5cd-b22fe006a94c  Southwestern Division        311
fdc42ec5-213c-11ec-8d40-1aa8b530b339  Pacific Southwest Region      50
fdf08bf1-ec5a-11ea-b83f-6e0e43d29f74  Eastern Region               434
fea6aa00-f6a7-11ea-ab86-ea94e31cc891  South Atlantic Division      114
ff9de9f8-ea5d-11eb-a9f1-725a14274964  Idaho (BLM)                    9
Length: 4698, dtype: int64


The above analysis suggests that Each Region Description has multiple FacilityIds

In [459]:
#calculate  how many facilityid are associated with each Sitetype
grouped_sitetype_per_facilityid= usa_camping_data_23.groupby(['facilityid','sitetype']).size()
print(grouped_sitetype_per_facilityid)

facilityid                            sitetype                         
72192                                 Hiking Zone                          6261
119090                                GROUP STANDARD NONELECTRIC            175
                                      STANDARD NONELECTRIC                 1582
119140                                RV ELECTRIC                           164
                                      STANDARD NONELECTRIC                 1728
                                                                           ... 
fdc42ec5-213c-11ec-8d40-1aa8b530b339  Centimudi Boat Launch Use Fee          47
fea6aa00-f6a7-11ea-ab86-ea94e31cc891  USACE Annual Day Use Pass              25
                                      USACE Day Use Pass                     89
ff9de9f8-ea5d-11eb-a9f1-725a14274964  Menan Day Use Pass                      6
                                      Menan Season Pass A (Single Pass)       3
Length: 7254, dtype: int64


In [460]:
grouped_sitetype_per_facilityid.head()

facilityid  sitetype                  
72192       Hiking Zone                   6261
119090      GROUP STANDARD NONELECTRIC     175
            STANDARD NONELECTRIC          1582
119140      RV ELECTRIC                    164
            STANDARD NONELECTRIC          1728
dtype: int64

We can conclude that one facilityid has multiple sitetypes

In [461]:
#calculate  how many facilityid are associated with each Sitetype
grouped_facilityid_per_sitetype= usa_camping_data_23.groupby(['sitetype','facilityid']).size()
print(grouped_facilityid_per_sitetype)

sitetype                                        facilityid                          
 Afton Landing Boat Ramp - Annual Day Use Pass  7916863d-8cfb-11ec-bbd9-5a6d94e5bf4b       41
1 Day                                           0a9802ee-aa0d-11ec-b367-827b1bb925c0       93
1 Day Parking Tag                               AP2510                                  27849
1 Day Pass                                      AP21                                        3
                                                AP22                                        4
                                                                                        ...  
Yes                                             251353                                     78
Young's Landing Annual Day Use Pass             AP21043                                     5
Young's Landing Day Use Pass                    AP21043                                    14
Zone                                            233273               

In [462]:
duplicates = grouped_facilityid_per_sitetype[grouped_facilityid_per_sitetype.duplicated()]
print(duplicates)

sitetype                             facilityid
1 Day Pass                           AP2266           3
1 Day Pass (S&P)                     AP21             4
1 Day Vehicle Pass                   AP2367           2
                                     AP2368           2
Angel Lake Picnic Area Day Use       AP21546          6
                                                   ... 
Yes                                  251353        1085
                                     251353          78
Young's Landing Annual Day Use Pass  AP21043          5
Young's Landing Day Use Pass         AP21043         14
Zone                                 251865          88
Length: 5406, dtype: int64


In [463]:
duplicates_sitetypes = grouped_sitetype_per_facilityid[grouped_sitetype_per_facilityid.duplicated()]
print(duplicates_sitetypes)

facilityid                            sitetype                         
121440                                STANDARD NONELECTRIC                 1728
231852                                GROUP STANDARD AREA NONELECTRIC        44
231879                                GROUP SHELTER ELECTRIC                 45
231894                                WALK TO                                12
231897                                STANDARD NONELECTRIC                  250
                                                                           ... 
fdc42ec5-213c-11ec-8d40-1aa8b530b339  Centimudi Boat Launch Use Fee          47
fea6aa00-f6a7-11ea-ab86-ea94e31cc891  USACE Annual Day Use Pass              25
                                      USACE Day Use Pass                     89
ff9de9f8-ea5d-11eb-a9f1-725a14274964  Menan Day Use Pass                      6
                                      Menan Season Pass A (Single Pass)       3
Length: 5406, dtype: int64


In [464]:
#calculate how many facilityid are associated with each equipmentdescription
grouped_equipmentdescription_per_facilityid= usa_camping_data_23.groupby(['facilityid','equipmentdescription']).size()
print(grouped_equipmentdescription_per_facilityid)

facilityid  equipmentdescription
119090      Tent                      3
            Trailer                   7
119140      RV                      107
            Tent                      2
            Trailer                   1
                                   ... 
274721      Pickup Camper             2
            Pop up                    2
            Tent                     52
72417       Tent                    248
            Trailer                 243
Length: 11046, dtype: int64


#Find the missing Values for each column.
1. Goal is to bring one model per site
2. Find NULL percentage for regions, Site type
3. Time Resolution - Is the information daily or weekly or Monthly
4. Number of dimensions- Number of regions, number of sites, data timeline is collected daily, weekly, monthly?

output from the recording
1. Find the NULL values (missing data) per site - meaning how many other columns are missing per site.
2. How many Sites per region. (Lets find how many camping - inventory )
3. Produce a df as region, site, all columns in dataset , number of missing values for that columns per region per site.
4. What columns are categorical what are not categorical..?
5. categorical columns means things that refer to classes.

In [465]:
# Lets calculate how many Inventory types per region
grouped_inventorytype_per_region= usa_camping_data_23.groupby(['regiondescription','inventorytype']).size()
print(grouped_inventorytype_per_region)

regiondescription  inventorytype     
Alaska             CAMPING                1415
                   TICKET                  258
Alaska Region      CAMPING               23745
                   PERMIT                 3199
                   TICKET                 3171
                                         ...  
Utah               LOTTERY_PERMIT        23073
                   PERMIT                 8132
                   VENUE_RESERVATIONS       16
Washington         CAMPING                2050
                   DONATION                  1
Length: 173, dtype: int64


In [466]:
# create a dataframe for grouping by region and inventory type
grouped_region_by_inventorytype_df = usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])
grouped_region_by_inventorytype_df.head(6)
grouped_region_by_inventorytype_create_df = pd.DataFrame(grouped_region_by_inventorytype_df)
grouped_region_by_inventorytype_create_df.to_csv("grouped_region_by_inventorytype_create_df.csv")

Create dataframes by grouping on Region and then Inventory type , further on calculate the missing percentage, min,max and std for each numerical columns.

In [467]:

def count_with_nan(x):
    return x.isna().mean() * 100

df_totalbeforetax_agg= usa_camping_data_23[['regiondescription', 'inventorytype', 'totalbeforetax']].head(100000).groupby(by=['regiondescription', 'inventorytype']).agg(
           totalbeforetax_isnull=('totalbeforetax', count_with_nan),
           totalbeforetax_min=('totalbeforetax', 'min'),
           totalbeforetax_max=('totalbeforetax', 'max'),
           totalbeforetax_std=('totalbeforetax', 'std'))
print(df_totalbeforetax_agg)

                                   totalbeforetax_isnull  totalbeforetax_min  \
regiondescription inventorytype                                                
Alaska            CAMPING                            0.0                 0.0   
                  TICKET                             0.0                 0.0   
Arizona           ACTIVITYPASS                       0.0                 0.0   
                  CAMPING                            0.0                 0.0   
                  DONATION                           0.0                 0.1   
                  LOTTERY_PERMIT                     0.0                 0.0   
                  PERMIT                             0.0                 0.0   
Jonestopia        CAMPING                            0.0                 0.0   
                  LOTTERY_CAMPING                    0.0                 0.0   

                                   totalbeforetax_max  totalbeforetax_std  
regiondescription inventorytype            

In [468]:
df_number_of_people_agg= usa_camping_data_23[['regiondescription', 'inventorytype', 'numberofpeople']].head(100000).groupby(by=['regiondescription', 'inventorytype']) .agg(
           numberofpeople_isnull=('numberofpeople', count_with_nan),
           numberofpeople_min=('numberofpeople', 'min'),
           numberofpeople_max=('numberofpeople', 'max'),
           numberofpeople_std=('numberofpeople', 'std'))

print(df_number_of_people_agg)

                                   numberofpeople_isnull  numberofpeople_min  \
regiondescription inventorytype                                                
Alaska            CAMPING                       0.000000                 0.0   
                  TICKET                        0.000000                 0.0   
Arizona           ACTIVITYPASS                100.000000                 NaN   
                  CAMPING                       0.000000                 1.0   
                  DONATION                    100.000000                 NaN   
                  LOTTERY_PERMIT              100.000000                 NaN   
                  PERMIT                        0.014501                 1.0   
Jonestopia        CAMPING                       0.000000                 0.0   
                  LOTTERY_CAMPING             100.000000                 NaN   

                                   numberofpeople_max  numberofpeople_std  
regiondescription inventorytype            

In [469]:
df_legacyfacilityid_agg= usa_camping_data_23[['regiondescription', 'inventorytype', 'legacyfacilityid']].head(100000).groupby(by=['regiondescription', 'inventorytype']) .agg(
           legacyfacilityid_isnull=('legacyfacilityid', count_with_nan),
           legacyfacilityid_min=('legacyfacilityid', 'min'),
           legacyfacilityid_max=('legacyfacilityid', 'max'),
           legacyfacilityid_std=('legacyfacilityid', 'std'))

print(df_legacyfacilityid_agg)

                                   legacyfacilityid_isnull  \
regiondescription inventorytype                              
Alaska            CAMPING                         0.000000   
                  TICKET                        100.000000   
Arizona           ACTIVITYPASS                  100.000000   
                  CAMPING                       100.000000   
                  DONATION                      100.000000   
                  LOTTERY_PERMIT                100.000000   
                  PERMIT                         61.803944   
Jonestopia        CAMPING                         0.000000   
                  LOTTERY_CAMPING                 0.000000   

                                   legacyfacilityid_min  legacyfacilityid_max  \
regiondescription inventorytype                                                 
Alaska            CAMPING                      146090.0              146090.0   
                  TICKET                            NaN                   

Create dataframes by grouping on Region and then Inventory type , further unique values of categorical columns and there sum.

In [470]:
# Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of sitetype
counts_unique_sitetype_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['sitetype'].value_counts())
counts_unique_sitetype_df.head(100)
#counts_unique_sitetype_df.to_csv ("counts_unique_sitetype_for region_inventorytype_df.csv")

count
regiondescription        inventorytype sitetype                              
Alaska                   CAMPING       CABIN NONELECTRIC                 1415
                         TICKET        Nature Tour                        133
                                       Historic Tour                       67
                                       Hike                                11
Alaska Region            CAMPING       STANDARD NONELECTRIC             10959
...                                                                       ...
Great Lakes / Ohio River CAMPING       GROUP WALK TO                       53
                                       EQUESTRIAN NONELECTRIC              30
                                       GROUP STANDARD NONELECTRIC          28
                                       GROUP STANDARD AREA NONELECTRIC      3
                         TICKET        Historic Tour                      619

[100 rows x 1 columns]

In [471]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of usetype
counts_unique_usetype_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['usetype'].value_counts())
counts_unique_usetype_df.head(10)
#counts_unique_usetype_df.to_csv ("counts_unique_usetype_for region_inventorytype_df.csv")

count
regiondescription inventorytype usetype         
Alaska            CAMPING       Overnight   1415
Alaska Region     CAMPING       Overnight  23744
Arizona           ACTIVITYPASS  Activity   11197
                  CAMPING       Overnight    898
California        ACTIVITYPASS  Activity     111
Colorado          ACTIVITYPASS  Activity       1
                  CAMPING       Overnight  14526
Colorado (BLM)    CAMPING       Overnight   1345
Eastern Region    ACTIVITYPASS  Activity    1370
                  CAMPING       Overnight  87590

In [472]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of facilitystate
counts_unique_facilitystate_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['facilitystate'].value_counts())
counts_unique_facilitystate_df.head(10)
#counts_unique_facilitystate_df.to_csv ("counts_unique_facilitystate_for region_inventorytype_df.csv")

count
regiondescription inventorytype      facilitystate        
Alaska            CAMPING            Alaska           1415
Alaska Region     CAMPING            Alaska          23709
                  PERMIT             Alaska           3199
                  VENUE_RESERVATIONS Alaska            440
Arizona           ACTIVITYPASS       Utah            11197
                  CAMPING            Arizona           898
                  LOTTERY_PERMIT     Arizona        135325
                  PERMIT             Arizona         11029
California        ACTIVITYPASS       California        111
                  PERMIT             California       6261

In [473]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of orderdate
counts_unique_orderdate_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['orderdate'].value_counts())
counts_unique_orderdate_df.head(100)
#counts_unique_orderdate_df.to_csv ("counts_unique_orderdate_for region_orderdate_df.csv")

count
regiondescription inventorytype orderdate                         
Alaska            CAMPING       2022-12-15T06:12:55.774219Z      4
                                2023-02-13T23:37:43.559799Z      4
                                2023-03-04T20:13:14.164698Z      4
                                2023-03-06T17:05:50.976476Z      4
                                2022-10-25T22:00:20.351754Z      3
...                                                            ...
                                2023-03-03T06:53:24.691847Z      2
                                2023-03-03T17:52:26.944788Z      2
                                2023-03-04T23:21:15.971799Z      2
                                2023-03-06T17:52:21.837262Z      2
                                2023-03-09T01:01:14.911053Z      2

[100 rows x 1 columns]

In [474]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of sitetype
counts_unique_sitetype_orderdate_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'sitetype'])['orderdate'].value_counts())
counts_unique_sitetype_orderdate_df.head(100)
#counts_unique_sitetype_df.to_csv ("counts_unique_orderdate_for region_sitetype_df.csv")

count
regiondescription sitetype          orderdate                         
Alaska            CABIN NONELECTRIC 2022-12-15T06:12:55.774219Z      4
                                    2023-02-13T23:37:43.559799Z      4
                                    2023-03-04T20:13:14.164698Z      4
                                    2023-03-06T17:05:50.976476Z      4
                                    2022-10-25T22:00:20.351754Z      3
...                                                                ...
                                    2023-03-03T06:53:24.691847Z      2
                                    2023-03-03T17:52:26.944788Z      2
                                    2023-03-04T23:21:15.971799Z      2
                                    2023-03-06T17:52:21.837262Z      2
                                    2023-03-09T01:01:14.911053Z      2

[100 rows x 1 columns]

In [475]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of nights
counts_unique_nights_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['nights'].value_counts())
counts_unique_nights_df.head(100)
#counts_unique_nights_df.to_csv ("counts_unique_nights_for region_inventorytype_df.csv")

count
regiondescription inventorytype nights        
Alaska            CAMPING       1 day     1269
                                2 days     124
                                3 days      22
                  TICKET        0          258
Alaska Region     CAMPING       1 day     9937
...                                        ...
Colorado          CAMPING       13 days      8
                                10 days      5
                                14 days      5
                                11 days      3
                                12 days      3

[100 rows x 1 columns]

In [476]:
#Explore creating dataframes by grouping on Region and Inventory type and counting the frequency for unique values of nights
counts_regions_inventorytype_orderdate_df = pd.DataFrame(usa_camping_data_23.groupby(['regiondescription', 'inventorytype'])['orderdate'].value_counts())
counts_regions_inventorytype_orderdate_df.head(100)
#counts_unique_nights_df.to_csv ("counts_unique_nights_for region_inventorytype_df.csv")

count
regiondescription inventorytype orderdate                         
Alaska            CAMPING       2022-12-15T06:12:55.774219Z      4
                                2023-02-13T23:37:43.559799Z      4
                                2023-03-04T20:13:14.164698Z      4
                                2023-03-06T17:05:50.976476Z      4
                                2022-10-25T22:00:20.351754Z      3
...                                                            ...
                                2023-03-03T06:53:24.691847Z      2
                                2023-03-03T17:52:26.944788Z      2
                                2023-03-04T23:21:15.971799Z      2
                                2023-03-06T17:52:21.837262Z      2
                                2023-03-09T01:01:14.911053Z      2

[100 rows x 1 columns]

In [477]:
#Lets find how the data is organized with respect to time. Find the difference between orderdate and starttime and group by region description and inventory type
usa_camping_data_23['order_timestamp'] = pd.to_datetime(usa_camping_data_23['orderdate'], errors='coerce')
usa_camping_data_23['start_timestamp'] = pd.to_datetime(usa_camping_data_23['startdate'], errors='coerce')
usa_camping_data_23['difference'] = (usa_camping_data_23['start_timestamp'] - usa_camping_data_23['order_timestamp']).dt.days
#print(usa_camping_data_23['difference'].head(10))

# Group by 'Category' and count occurrences of each unique difference
difference_count = usa_camping_data_23.groupby(['regiondescription', 'inventorytype']).agg(
           difference_isnull=('difference', count_with_nan),
           difference_min=('difference', 'min'),
           difference_max=('difference', 'max'),
           difference_mean=('difference', 'mean'),
           difference_std=('difference', 'std'))
print(difference_count)

                                      difference_isnull  difference_min  \
regiondescription inventorytype                                           
Alaska            CAMPING                           0.0            -2.0   
                  TICKET                            0.0            -2.0   
Alaska Region     CAMPING                           0.0            -2.0   
                  PERMIT                            0.0           -23.0   
                  TICKET                            0.0            -2.0   
...                                                 ...             ...   
Utah              LOTTERY_PERMIT                    0.0            72.0   
                  PERMIT                            0.0            -1.0   
                  VENUE_RESERVATIONS                0.0             3.0   
Washington        CAMPING                           0.0             0.0   
                  DONATION                        100.0             NaN   

                        

All the code below is filtered for camping and lottery camping Inventory type.


In [478]:
#Lets Create a dataframe and filter data for camping and lottery camping for all regions
camping_inventory_df = usa_camping_data_23[usa_camping_data_23['inventorytype'].isin(['CAMPING', 'LOTTERY_CAMPING'])]
camping_inventory_df.head(10)
camping_inventory_df.describe()
camping_inventory_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4125599 entries, 0 to 8980485
Data columns (total 38 columns):
 #   Column                   Dtype              
---  ------                   -----              
 0   historicalreservationid  object             
 1   ordernumber              object             
 2   agency                   object             
 3   orgid                    int64              
 4   codehierarchy            object             
 5   regioncode               object             
 6   regiondescription        object             
 7   parentlocationid         object             
 8   parentlocation           object             
 9   legacyfacilityid         float64            
 10  park                     object             
 11  sitetype                 object             
 12  usetype                  object             
 13  productid                object             
 14  inventorytype            object             
 15  facilityid               object      

In [479]:
num_rows = len(camping_inventory_df)
print(num_rows)


4125599


In [480]:
# Create a csv File from a dataframe for camping and Lottery Camping (Inventory type) for all regions.
''''
camping_inventory_df.to_csv("All_regions_camping.csv")'''

'\'\ncamping_inventory_df.to_csv("All_regions_camping.csv")'

In [481]:
# Produce a df as region, site, all columns in dataset, number of missing values for that columns per region per site

columns_of_interest = ['parentlocationid', 'parentlocation',
       'park', 'sitetype', 'usetype', 'productid', 'inventorytype',
       'facilityid', 'facilitystate', 'customerzip', 'totalpaid', 'startdate',
       'enddate', 'orderdate', 'nights', 'equipmentdescription']
missing_counts = camping_inventory_df[columns_of_interest].isnull().groupby(camping_inventory_df['regiondescription']).sum()
print(missing_counts)


                                      parentlocationid  parentlocation  park  \
regiondescription                                                              
Alaska                                               0               0     0   
Alaska Region                                        0               0     0   
Arizona                                              0               0     0   
Colorado                                             0               0     0   
Colorado (BLM)                                       0               0     0   
Eastern Region                                       0               0     0   
Great Lakes / Ohio River                             0               0     0   
Great Plains                                         0               0     0   
Idaho (BLM)                                          0               0     0   
Intermountain Region                                 0               0     0   
Jonestopia                              

In [482]:
missing_any_count = camping_inventory_df.groupby(['regiondescription', 'inventorytype']).apply(lambda x: x[['parentlocationid', 'parentlocation',
       'park', 'sitetype', 'usetype', 'productid', 'inventorytype',
       'facilityid', 'facilitystate', 'customerzip', 'totalpaid', 'startdate',
       'enddate', 'orderdate', 'nights', 'equipmentdescription']].isnull().any(axis=1).sum())
print(missing_any_count)



regiondescription                     inventorytype  
Alaska                                CAMPING              1415
Alaska Region                         CAMPING             12191
Arizona                               CAMPING               103
Colorado                              CAMPING             11419
Colorado (BLM)                        CAMPING              1345
Eastern Region                        CAMPING             27339
Great Lakes / Ohio River              CAMPING             37514
Great Plains                          CAMPING              1721
Idaho (BLM)                           CAMPING               264
Intermountain Region                  CAMPING            131923
Jonestopia                            CAMPING                41
                                      LOTTERY_CAMPING         3
Mid Pacific                           CAMPING              4814
Midwest Region                        CAMPING             33911
Mississippi Valley                    CAMPING     

/var/folders/ls/mlx1cxs96mx4gpcd6hv_g4f00000gn/T/ipykernel_35812/1760748887.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_any_count = camping_inventory_df.groupby(['regiondescription', 'inventorytype']).apply(lambda x: x[['parentlocationid', 'parentlocation',


In [483]:
df = pd.DataFrame(camping_inventory_df)

# Define the columns you want to check for missing values
columns_to_check = ['parentlocationid', 'parentlocation',
       'park', 'sitetype', 'usetype', 'productid',
       'facilityid', 'facilitystate', 'customerzip', 'totalpaid', 'startdate',
       'enddate', 'orderdate', 'nights', 'equipmentdescription']

# Create a mask for missing values in these columns
missing_mask = df[columns_to_check].isnull()

# Group by 'regiondescription' and 'inventorytype' and calculate the percentage of missing values
missing_percentage_for_camping_inventory = (
    missing_mask.groupby([df['regiondescription'], df['inventorytype']])
    .mean() * 100
)

print(missing_percentage_for_camping_inventory)
missing_percentage_for_camping_inventory.head(5)

                                                      parentlocationid  \
regiondescription                    inventorytype                       
Alaska                               CAMPING                       0.0   
Alaska Region                        CAMPING                       0.0   
Arizona                              CAMPING                       0.0   
Colorado                             CAMPING                       0.0   
Colorado (BLM)                       CAMPING                       0.0   
Eastern Region                       CAMPING                       0.0   
Great Lakes / Ohio River             CAMPING                       0.0   
Great Plains                         CAMPING                       0.0   
Idaho (BLM)                          CAMPING                       0.0   
Intermountain Region                 CAMPING                       0.0   
Jonestopia                           CAMPING                       0.0   
                                     L

,,parentlocationid,parentlocation,park,sitetype,usetype,productid,facilityid,facilitystate,customerzip,totalpaid,startdate,enddate,orderdate,nights,equipmentdescription
regiondescription,inventorytype,,,,,,,,,,,,,,,
Alaska,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,36.961131,0.0,0.0,0.000000,0.0,0.0,100.000000
Alaska Region,CAMPING,0.0,0.0,0.0,0.004211,0.004211,0.0,0.0,0.151611,20.408507,0.0,0.0,0.004211,0.0,0.0,38.429143
Arizona,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,11.469933,0.0,0.0,0.000000,0.0,0.0,0.000000
Colorado,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,20.983065,0.0,0.0,0.000000,0.0,0.0,72.057001
Colorado (BLM),CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,18.810409,0.0,0.0,0.000000,0.0,0.0,100.000000


In [484]:
missing_percentage_for_camping_inventory.head(5)

,,parentlocationid,parentlocation,park,sitetype,usetype,productid,facilityid,facilitystate,customerzip,totalpaid,startdate,enddate,orderdate,nights,equipmentdescription
regiondescription,inventorytype,,,,,,,,,,,,,,,
Alaska,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,36.961131,0.0,0.0,0.000000,0.0,0.0,100.000000
Alaska Region,CAMPING,0.0,0.0,0.0,0.004211,0.004211,0.0,0.0,0.151611,20.408507,0.0,0.0,0.004211,0.0,0.0,38.429143
Arizona,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,11.469933,0.0,0.0,0.000000,0.0,0.0,0.000000
Colorado,CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,20.983065,0.0,0.0,0.000000,0.0,0.0,72.057001
Colorado (BLM),CAMPING,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,18.810409,0.0,0.0,0.000000,0.0,0.0,100.000000


In [485]:
#Create a dataframe for missing_percentage_for_camping_inventory
missing_percentage_for_camping_inventory_df = pd.DataFrame(missing_percentage_for_camping_inventory)
#Create a csv file for missing_percentage_for_camping_inventory
missing_percentage_for_camping_inventory.to_csv("missing_percentage_for_camping_inventory.csv")

In [486]:
#Lets find how the data is organized with respect to time. Find the difference between orderdate and starttime and group by 
# region description and inventory type for only camoing inventory dataframe instead of entire dataset with all inventory types as above.
camping_inventory_df['order_timestamp'] = pd.to_datetime(camping_inventory_df['orderdate'], errors='coerce')
camping_inventory_df['start_timestamp'] = pd.to_datetime(usa_camping_data_23['startdate'], errors='coerce')
camping_inventory_df['difference'] = (camping_inventory_df['start_timestamp'] - camping_inventory_df['order_timestamp']).dt.days
#print(usa_camping_data_23['difference'].head(10))

# Group by 'Category' and count occurrences of each unique difference
difference_count_camping = camping_inventory_df.groupby(['regiondescription', 'inventorytype']).agg(
           difference_mean=('difference', 'mean'),
           difference_std=('difference', 'std'))
print(difference_count_camping)

/var/folders/ls/mlx1cxs96mx4gpcd6hv_g4f00000gn/T/ipykernel_35812/1354887556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  camping_inventory_df['order_timestamp'] = pd.to_datetime(camping_inventory_df['orderdate'], errors='coerce')
/var/folders/ls/mlx1cxs96mx4gpcd6hv_g4f00000gn/T/ipykernel_35812/1354887556.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  camping_inventory_df['start_timestamp'] = pd.to_datetime(usa_camping_data_23['startdate'], errors='coerce')
/var/folders/ls/mlx1cxs96mx4gpcd6hv_g4f

                                                      difference_mean  \
regiondescription                    inventorytype                      
Alaska                               CAMPING                20.054417   
Alaska Region                        CAMPING                90.446705   
Arizona                              CAMPING                19.456570   
Colorado                             CAMPING                31.935013   
Colorado (BLM)                       CAMPING                34.098141   
Eastern Region                       CAMPING                80.006248   
Great Lakes / Ohio River             CAMPING                74.955358   
Great Plains                         CAMPING                70.481726   
Idaho (BLM)                          CAMPING                58.624049   
Intermountain Region                 CAMPING                71.251635   
Jonestopia                           CAMPING                24.682927   
                                     LOTTERY_CAMPIN

Summary
1. There is booking data for 50 states from recreation.gov website for year 2023. It includes total 35 columns and about 8984097 rows. 
   Various columns associated are regions description, Parent location Ids, Site types,USer type, inventory typefacility Ids, equipment  
   descriptions, number of people visited, total paid and many more.
2. Total there are 22 categorical and 13 non-categorical columns.
3. Each column has either one to one or one-many relationship associated with each other. eg. Region code and Region Description are one to 
   one relation. Multiple Facility Ids are associated with each facility state.
4. Few columns have more that 50% data missing eg. equipment descrition has 57% while some like sitetype and number of people around 36-35%. 
   This suggests how we can use those columns and fill in (assume) data for missing ones.
5. we also calculated how many sites are associated with each region.
6. we also calculated how many equipment descriptions are associated with each region.
7. we also calculated how many regions are associated with each facility state.
8. Created df by Calculating the missing values per region for each inventory type
9. Created df by Calculating the missing values per region for each site type
10. What columns are categorical what not categorical
11. Created dataframes to calculate the (null,min,max,stdev) for  numerical columns (eg number of people) per region 
    per inventory type (eg camping)
12. Created dataframes and csv files for categorical columns to count the number of eg. site types per region and per inventory type
13. Created dataframes and csv files for categorical columns to count the number of eg. facility state per region and per inventory type.
14. For our goal of time series forecasting we need to understand how the data is related with the time. eg frequency of data, weekly, 
    monthly etc. We have order date, start date and end date for bookings in our dataset. I have developed feature by taking difference 
    between start time and order date and calculated min, max, mean and stdev for the difference value. The mean is suggesting how each 
    region has a diverse difference in the timelines between there order dates and startdays  eg. Alaska has 20 days while Alaska Region has 
    about 90 days. Narrowing down this method to only camping in different regions provides a better idea that every region has a difference 
    in the way orders are made before the actual start date of camp day.
